In [1]:
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.express as px

In [2]:
import numpy as np
import pandas as pd

In [3]:
import pulp

In [4]:
f_conv_mon = 0.0008

In [5]:
def bmatrix(a):
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [6]:
C = np.array([[105, 1267, 1134, 2758, 515, 1350, 1311, 1750, 1565, 1137, 3110, 1365]]) #dinar

C = C * f_conv_mon

C = np.concatenate((C[:, 0:1], C[:, 2:4], C[:, 5:11]), axis=1)

C = np.concatenate((C, C), axis=1)


In [7]:
A = np.array([  [     1,       1,       1,       1,       1,       1,       0,       0,       0,       0,       0,       0],
                [     0,       0,       0,       0,       0,       0,       1,       1,       1,       1,       1,       1],
                [   100,      50,      50,      50,     100,     100,      50,     100,     100,     100,     100,     100],
                [   100,      50,      50,      25,      50,     150,      50,     100,     100,      50,      50,      50],
                [     0,       0,       0,       0,       0,      12,       0,       0,      12,       0,       0,       0],
                [     1,       1,       2,       1,       1,       3,       1,       2,       2,       2,       3,       1],
                [     6,      29,      32,      14,      54,      60,       0,       0,      18,      13,      29,       8],
                [     2,       0,      17,      23,       0,       0,      22,       6,      50,     102,      77,      31],
                [     0,       0,       0,       0,       0,      12,      35,      24,      80,       0,     112,      58],
                [     9,       8,       7,      34,      26,      22,       0,       0,      52,       0,      43,       0],
                [   1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5],
                [     1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1],
                [   0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5],
                [   0.5,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0],
                [   390,     355,     370,     430,     440,       0,     275,       0,     100,     133,     213,     100],
                [   275,       0,     470,     305,       0,       0,    1430,     188,    1015,     955,    1343,    1215],
                [     0,       0,     150,       0,       0,     750,     443,     840,    1683,       0,    1748,    1095],
                [   300,     295,     305,     303,     210,     860,       0,       0,       0,       0,       0,       0],
                [345000,  233000,  216000,  242000,  360000,  650000,  244000,  242000,  310000,  263000,  250000,  285000]])

In [8]:
A[18,:] = A[18,:] * f_conv_mon

In [9]:
A = np.concatenate((A[:, 0:1], A[:, 2:4], A[:, 5:11]), axis=1)
A = np.concatenate((np.concatenate((A, np.zeros(A.shape)), axis=1), 
                    np.concatenate((np.zeros(A.shape), A), axis=1),
                    np.concatenate((-np.ones((1, 1)) ,  np.zeros((1, 9)),  np.ones((1, 2))  ,  np.zeros((1, 6))), axis=1),
                    np.concatenate(( np.zeros((1, 1)), -np.ones((1, 2)) ,  np.zeros((1, 11)),  np.ones((1, 1))  , np.zeros((1, 3))), axis=1)
                    ), axis=0)

In [10]:
np.set_printoptions(precision = 3, suppress = True)
print(bmatrix(A[38:40, 0:100]))

\begin{bmatrix}
  -1. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 1. & 1. & 0. & 0. & 0. & 0. & 0. & 0.\\
  0. & -1. & -1. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 0. & 1. & 0. & 0. & 0.\\
\end{bmatrix}


In [11]:
b = np.array([2395,2260,419190,397560,7000,13075,50593,554800,562400,562400,103056,103056,103056,954,20699716,20929709,21159705,21159705,1445855000]) #dinar

b[18] = b[18] * f_conv_mon

In [12]:
b = np.concatenate((b, b, np.array([0, 0])), axis=0)

In [13]:
def create_var():
    var = np.array([])

    lbx = [0] * 18
    lbx[0] = 1500
    lbx[9] = 1500


    for i in range(1, 19):
        var = np.append(var, np.array([pulp.LpVariable("x{}".format(i), lowBound = lbx[i-1], upBound=None)]))
    
    return var

In [14]:
def objective():
    var = create_var()

    fo = C.dot(var)[0]

    return (var, fo)

In [15]:
def print_solution(sol, linearProblem, X):
    saida = "\nStatus = {} \nValue = {}".format(pulp.LpStatus[sol], pulp.value(linearProblem.objective))
    
    for i in range(0, X.shape[0]):
        saida += "\n{} = {}".format(X[i], pulp.value(X[i]))
    
    print(saida)

In [16]:
d_var, f = objective()
model = pulp.LpProblem("First_Model", pulp.LpMaximize)

model += f 

Ax = A.dot(d_var)

for i in range(0, Ax.shape[0]):
    model += Ax[i] <= b[i]

In [17]:
#model.writeLP('models\original_problem_model.lp')
solution = model.solve(pulp.GLPK(path='F:\glpk-4.65\w64\glpsol.exe', options=['--ranges models\sensit_original_2y.sen']))

In [18]:
print_solution(sol=solution, linearProblem=model, X=d_var)


Status = Optimal 
Value = 12571.7924864
x1 = 1500.0
x2 = 0.0
x3 = 895.0
x4 = 0.0
x5 = 31.2
x6 = 1226.63
x7 = 0.0
x8 = 0.0
x9 = 1002.17
x10 = 1500.0
x11 = 0.0
x12 = 895.0
x13 = 0.0
x14 = 362.828
x15 = 895.0
x16 = 0.0
x17 = 0.0
x18 = 1002.17


In [19]:
for v in model.variables():
    print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)

x1 = 1500.0 	Reduced Cost = None
x10 = 1500.0 	Reduced Cost = None
x11 = 0.0 	Reduced Cost = None
x12 = 895.0 	Reduced Cost = None
x13 = 0.0 	Reduced Cost = None
x14 = 362.828 	Reduced Cost = None
x15 = 895.0 	Reduced Cost = None
x16 = 0.0 	Reduced Cost = None
x17 = 0.0 	Reduced Cost = None
x18 = 1002.17 	Reduced Cost = None
x2 = 0.0 	Reduced Cost = None
x3 = 895.0 	Reduced Cost = None
x4 = 0.0 	Reduced Cost = None
x5 = 31.2 	Reduced Cost = None
x6 = 1226.63 	Reduced Cost = None
x7 = 0.0 	Reduced Cost = None
x8 = 0.0 	Reduced Cost = None
x9 = 1002.17 	Reduced Cost = None


In [20]:
print(bmatrix(C))

\begin{bmatrix}
  0.084 & 0.907 & 2.206 & 1.08 & 1.049 & 1.4 & 1.252 & 0.91 & 2.488 & 0.084 & 0.907 & 2.206\\
  1.08 & 1.049 & 1.4 & 1.252 & 0.91 & 2.488\\
\end{bmatrix}


In [21]:
#from unicodedata import decimal
#df = pd.DataFrame(b.transpose())
#df.to_clipboard(index=False,header=False, decimal=',')